In [1]:
import os, sys
import numpy as np
import pandas as pd
from difflib import SequenceMatcher
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope

import mlflow
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient

In [3]:
# charsAll = pd.read_csv('../data_raw/dnd_chars_all.tsv', sep = '\t')
charsAll = pd.read_csv("https://raw.githubusercontent.com/oganm/dnddata/master/data-raw/dnd_chars_all.tsv", sep = '\t')
display(charsAll)

,ip,finger,hash,name,race,background,date,class,justClass,subclass,...,choices,country,countryCode,processedAlignment,good,lawful,processedRace,processedSpells,processedWeapons,alias
0,NaN,ed15f9d,fe3ed6570067d2cd808bcee0a4396824,ee1e382c,Hill Dwarf,Guild Member - Justice,2022-08-23T20:02:11Z,Sorcerer 13|Cleric 1,Sorcerer|Cleric,Clockwork Soul|Order Domain,...,metamagic/Twinned Spell*Subtle Spell*Quickened...,Canada,CA,NaN,NaN,NaN,Dwarf,Alarm*1|Protection from Evil and Good*1|Comman...,"Crossbow, Light|Dagger",thirsty_davinci
1,NaN,ed15f9d,aa656cef94740b1da714d82782d8a8c4,ee1e382c,Hill Dwarf,Guild Member - Justice,2022-08-23T19:43:25Z,Sorcerer 13|Cleric 1,Sorcerer|Cleric,Clockwork Soul|Order Domain,...,metamagic/Twinned Spell*Subtle Spell*Quickened...,Canada,CA,NaN,NaN,NaN,Dwarf,Alarm*1|Protection from Evil and Good*1|Comman...,"Crossbow, Light|Dagger",thirsty_davinci
2,6b5d3f4,d922658,04b9abba9ce759ef13319c520bf0eb69,f1f6ff43,Human,Noble,2022-08-22T14:57:09Z,Fighter 13,Fighter,Eldritch Knight,...,fighting style/Defense,United States,US,CG,NaN,NaN,Human,Prestidigitation*0|Mage Hand*0|Blade Ward*0|Li...,"Longsword|Dagger|Crossbow, Light|Mace",cool_bhabha
3,9b7218f,b5d19a0,ba92372c716a832e03e866447cb2d102,f92bdd74,Fallen Aasimar,Outlander,2022-08-22T12:12:53Z,Sorcerer 3|Druid 2,Sorcerer|Druid,Shadow Magic|Circle of Wildfire,...,metamagic/Transmuted Spell*Seeking Spell,Brazil,BR,NaN,NaN,NaN,Aasimar,Burning Hands*1|Cure Wounds*1|Darkness*2,Dagger,vigilant_mcclintock
4,9b7218f,b5d19a0,2f4a637dd42b9aa945abf870fe34edcf,f92bdd74,Fallen Aasimar,Outlander,2022-08-22T12:07:21Z,Sorcerer 3|Druid 2,Sorcerer|Druid,Shadow Magic|Circle of Wildfire,...,metamagic/Transmuted Spell*Seeking Spell,Brazil,BR,NaN,NaN,NaN,Aasimar,Burning Hands*1|Cure Wounds*1|Darkness*2,Dagger,vigilant_mcclintock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10889,d580ce9,d580ce9,d936585b5ca49f41c57022f5c7cb4ab8,6f906508,Tiefling,Hermit,2018-04-07T19:49:23Z,Monk 3,Monk,Way of the Drunken Master,...,NaN,NaN,NaN,NaN,NaN,NaN,Tiefling,NaN,Quarterstaff|Dart|Unarmed Strike,nervous_goldstine
10890,d580ce9,d580ce9,05998a5e27f3b8f14b58644ca9a10d42,bcf1fa28,Half-Elf,Mercenary Veteran,2018-04-07T15:40:55Z,Ranger 3,Ranger,Beast Master,...,fighting style/Archery,NaN,NaN,NaN,NaN,NaN,Half-Elf,NaN,"Shortsword|Crossbow, Light",gallant_hugle
10891,d580ce9,d580ce9,c224994d1134d58ae83dd5618ce192e9,5fcc1db9,Dragonborn,Noble,2018-04-07T09:21:55Z,Paladin 6,Paladin,Oath of Ancients,...,fighting style/Great Weapon Fighting,NaN,NaN,CG,NaN,NaN,Dragonborn,Ensnaring Strike*1|Speak with Animals*1|Cure W...,Longsword|Javelin,condescending_lederberg
10892,d580ce9,d580ce9,3ea236dcfd2ecde78718881fe8e619b6,acc2a5e1,Meadowguard,Acolyte,2018-04-07T08:04:10Z,Bard 1,Bard,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mending*0|Light*0|Minor Illusion*0|Heroism*1|H...,Dagger|Dagger,relaxed_cori


In [3]:
charsUnique = pd.read_csv('../data_raw/dnd_chars_unique.tsv', sep = '\t')
display(charsUnique.tail())

,ip,finger,hash,name,race,background,date,class,justClass,subclass,...,choices,country,countryCode,processedAlignment,good,lawful,processedRace,processedSpells,processedWeapons,alias
7941,b78f544,d335886,fa01f62e2c2d0f656a63544cb87dbefe,2452ffc4,Protector Aasimar,Haunted One,2020-12-19T20:45:38Z,Paladin 1|Warlock 1,Paladin|Warlock,|The Hexblade,...,NaN,United States,US,NaN,NaN,NaN,Aasimar,Eldritch Blast*0|Green-Flame Blade*0|Hex*1|Hel...,"Maul|Crossbow, Heavy|Javelin|Longsword",jolly_diffie
7942,d74b2dc,ec782f0,c29a7237831581f759743c8185aa288b,8185b557,Tiefling,Entertainer,2020-10-05T00:08:43Z,Paladin 1|Warlock 1,Paladin|Warlock,|The Fiend,...,tiefling variants menu/Devil's Tongue,United States,US,LE,NaN,NaN,Tiefling,NaN,Morningstar|Mace,romantic_mirzakhani
7943,6cda65b,13bd3af,3d3399420ed636fdc7bb623edb8ab2f7,cdf56fad,Human,Sage,2020-10-02T23:36:18Z,Cleric 1|Warlock 1,Cleric|Warlock,Grave Domain|The Great Old One,...,NaN,Canada,CA,NN,NaN,NaN,Human,Guidance*0|Light*0|Sacred Flame*0|Spare the Dy...,"Mace|Crossbow, Light",brave_moser
7944,0ccab57,NaN,e8d9dd6e8558879a2837304bfbdeba78,c0115930,Half-Orc,Outlander,2020-06-04T16:11:16Z,Barbarian 1|Warlock 1,Barbarian|Warlock,|The Fiend,...,NaN,United States,US,CN,NaN,NaN,Half-Orc,NaN,Battleaxe|Greatclub|Javelin,jovial_heyrovsky
7945,123ce8d,96fda9f,96b8c11f2ca3d93da9d3c4b297554c65,54c8aae3,Wood Elf,Hermit,2020-04-12T00:04:32Z,Rogue 1|Monk 1,Rogue|Monk,|,...,expertise/Thieves' Tools*Stealth,United Kingdom,GB,NaN,NaN,NaN,Elf,NaN,Rapier|Shortbow|Dagger,dreamy_shirley


In [7]:
charsAll.columns

Index(['ip', 'finger', 'hash', 'name', 'race', 'background', 'date', 'class',
       'justClass', 'subclass', 'level', 'feats', 'HP', 'AC', 'Str', 'Dex',
       'Con', 'Int', 'Wis', 'Cha', 'alignment', 'skills', 'weapons', 'spells',
       'castingStat', 'choices', 'country', 'countryCode',
       'processedAlignment', 'good', 'lawful', 'processedRace',
       'processedSpells', 'processedWeapons', 'alias'],
      dtype='object')

In [4]:
# Function to identify the class with the most levels in and use that as the target
def dominantClass(classStr, justClass):
    i = 0
    totalLvl = 0
    if '|' not in classStr:
        if 'Revised Ranger' in classStr:
            return 'Ranger', int(classStr.split(' ')[-1])
        else:
            return classStr.split(' ')[0], int(classStr.split(' ')[-1])
    dClass = ''
    for c_lvl in classStr.split('|'):
        if 'Revised Ranger' in c_lvl:
            c = 'Ranger'
            lvl = c_lvl.split(' ')[-1]
        else:
            try: 
                c, lvl = c_lvl.split(' ')
            except ValueError as ve:
                c = SequenceMatcher(None, c_lvl, justClass).find_longest_match()
                lvl = c_lvl.split(' ')[-1]
        totalLvl += int(lvl)
        if int(lvl) > i:
            dClass = c
            i = int(lvl)
    return dClass, totalLvl

charsAll['target'], charsAll['level'] = zip(*charsAll[['class', 'justClass']].apply(lambda x: dominantClass(x['class'], x['justClass']), axis = 1))

In [5]:
targetCounts = charsAll.groupby('target')['hash'].agg('count').reset_index().rename(columns = {'hash': 'counts'})
display(targetCounts)

,target,counts
0,Artificer,218
1,Barbarian,943
2,Bard,736
3,Battle,1
4,Blood,24
5,Cleric,995
6,Crafting,1
7,Druid,728
8,Fighter,1441
9,Gunslinger,2


In [6]:
# Find and clean up data
# Remove entries with less than or equal to 2 submissions for that dominant class
features = ['HP', 'AC', 'Str', 'Dex', 'Con', 'Int', 'Wis', 'Cha', 'level']
target = 'target'
colList = features + [target]
dfDropNA = charsAll[colList].dropna(how = 'any')
realClasses = targetCounts[targetCounts['counts'] > 2]
cleanDf = dfDropNA.merge(realClasses['target'], how = 'inner', on = target)
display(cleanDf)

,HP,AC,Str,Dex,Con,Int,Wis,Cha,level,target
0,146,10,9,11,20,14,14,20,14,Sorcerer
1,133,10,9,11,18,14,14,20,14,Sorcerer
2,140,21,20,12,19,14,11,10,13,Fighter
3,34,16,8,10,14,10,16,16,5,Sorcerer
4,34,16,8,10,14,10,16,16,5,Sorcerer
...,...,...,...,...,...,...,...,...,...,...
10884,25,13,16,14,13,17,12,18,3,Monk
10885,32,16,10,16,14,13,13,10,3,Ranger
10886,58,19,20,13,15,10,12,16,6,Paladin
10887,10,14,8,16,13,11,13,16,1,Bard


In [13]:
# realClasses.to_dict()
rs_no = 33
train_test_split(cleanDf.loc[:, features], cleanDf.loc[:, target], test_size = 0.3, random_state = rs_no)

[      HP  AC  Str  Dex  Con  Int  Wis  Cha  level
 185   24  16   16    8   14   10   15   12      3
 6002  28  13   10   14    8   15   13   16      4
 6762  44  16   17   12   15   12   10   10      5
 8096   7  13   16   15    8   11    7   17      1
 2224  12  12   14   10   16   10   15   14      1
 ...   ..  ..  ...  ...  ...  ...  ...  ...    ...
 57    32  19   14   10   10   13   12   16      4
 578   49  12    8   14   16   10   18   20      8
 5848  24  14   13   16   12   10   10   16      4
 2439  30  14    9   16   13   11   13   19      4
 7188  27  17   10   18   12   11   16    8      4
 
 [7622 rows x 9 columns],
       HP  AC  Str  Dex  Con  Int  Wis  Cha  level
 3332  51  14   17   16   13   12   10   12      8
 3752  15  13   13   15   13   13   13   13      2
 5539  74  19   12   18   14   18   12   10      7
 4025   9  14   11   15   13   12   15    9      1
 5586  10  18   13   12   14    8   16   12      1
 ...   ..  ..  ...  ...  ...  ...  ...  ...    ...
 78

In [6]:
# pd.DataFrame({'target': {0: 'Artificer',  1: 'Barbarian',  2: 'Bard',  4: 'Blood',  5: 'Cleric',  7: 'Druid',  8: 'Fighter',  10: 'Monk',
#                          11: 'Mystic',  12: 'Paladin',  13: 'Ranger',  14: 'Rogue',  15: 'Sorcerer',  16: 'Warlock',  17: 'Wizard'},
#             'counts': {0: 218,  1: 943,  2: 736,  4: 24,  5: 995,  7: 728,  8: 1441,  10: 732,  11: 13,  12: 874,  13: 792,  14: 1292,
#                        15: 664,  16: 677,  17: 760}})
rs_no = 33
X_trainVal, X_test, y_trainVal, y_test = train_test_split(cleanDf.loc[:, features], cleanDf.loc[:, target], test_size = 0.3, random_state = rs_no)
X_train, X_val, y_train, y_val = train_test_split(X_trainVal, y_trainVal, test_size = 0.25, random_state = rs_no + 3)

In [13]:
rf = RandomForestClassifier(max_depth = 20, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)

In [14]:
accuracy_score(y_val, y_pred)

0.6768100734522561

In [6]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('HPO-DnD-classification')

def load_pickle(filename):
    with open(filename, 'rb') as f_in:
        return pickle.load(f_in)

def find_optimal(**kwargs):
    # kwargs -> {'data_path': str, 'num_trials': int}
    X_train, y_train = load_pickle(os.path.join(kwargs['data_path'], 'train.pkl'))
    X_val, y_val = load_pickle(os.path.join(kwargs['data_path'], 'val.pkl'))

    def objective(params):
        with mlflow.start_run():
            mlflow.set_tag('model', 'RandomForestClassifier')
            mlflow.log_params(params)
            rf = RandomForestClassifier(**params)
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_val)
            accScore = accuracy_score(y_val, y_pred)
            mlflow.log_metric('accuracy_score', accScore)
            mlflow.log_metric('loss', 1 - accScore)

        return {'loss': 1 - accScore, 'status': STATUS_OK}

    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        'random_state': 42
    }

    rstate = np.random.default_rng(42)  # for reproducible results
    fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=kwargs['num_trials'],
        trials=Trials(),
        rstate=rstate
    )

find_optimal(data_path = '/workspaces/DnDClassClassification/train_outputs/', num_trials = 10)

100%|██████████| 10/10 [00:04<00:00,  2.11trial/s, best loss: 0.3389296956977964]
